<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Aguirre
- Nombre de alumno 2: Ignacio Albornoz


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/tomasaguirre-ignacioalbornoz/MDS7202`

# Importamos librerias útiles

In [52]:
!pip install -qq xgboost optuna
!pip install joblib
!pip install plotly

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [53]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')


df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


Inspección de los datos:

In [54]:
df.describe()

,id,lat,long,pop,price,quantity
count,7456.000000,7456.000000,7456.000000,7456.000000,7456.000000,7456.000000
mean,3784.926770,38.300616,23.270170,355042.733637,1.197193,29408.428380
std,2185.822361,1.650030,1.086592,232336.703020,0.818175,17652.985675
min,0.000000,35.327870,21.734440,134219.000000,0.110000,2953.000000
25%,1889.750000,37.962450,22.417610,141732.000000,0.620000,16572.750000
50%,3783.500000,38.244440,22.930860,257501.500000,0.930000,25294.500000
75%,5682.250000,39.636890,23.716220,665102.000000,1.510000,37699.000000
max,7559.000000,40.643610,25.143410,672130.000000,4.790000,145287.000000


Revisión de datos nulos:

In [55]:
df.isna().sum()

id           0
date         0
city         0
lat          0
long         0
pop          0
shop         0
brand        0
container    0
capacity     0
price        0
quantity     0
dtype: int64

Gráfico de la matriz de correlación entre variables numéricas para ver si es es necesario eliminar variables.

In [56]:

import plotly.express as px

# Calcular la matriz de correlación
correlation_matrix = df.corr()

# Crear la figura con Plotly Express
fig = px.imshow(correlation_matrix, labels=dict(color='Correlación'), x=correlation_matrix.columns, y=correlation_matrix.columns, title= 'Matriz de correlación entre variables numéricas')

# Mostrar la figura
fig.show()



C:\Users\tomas\AppData\Local\Temp\ipykernel_16040\1329622135.py:4: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Solamente es necesario eliminar la columna de id para proceder con el laboratorio.

In [57]:
df.drop('id', inplace = True, axis = 1)

## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [58]:
# Importar librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib


ordered_columns = ['price'] + [col for col in df.columns if col != 'price']
df = df[ordered_columns]

# Crear un FunctionTransformer para extraer día, mes y año
def extract_date_parts(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    df = df.drop(columns=['date'])
    return df



def get_feature_names(column_transformer):
    """Get feature names from a ColumnTransformer."""
    output_features = []

    for name, pipe, features in column_transformer.transformers_:
        # Process each transformer
        if name != 'remainder':
            if hasattr(pipe, 'get_feature_names_out'):
                # If the transformer has a get_feature_names_out method, use it
                feature_names = pipe.get_feature_names_out(features)
                output_features.extend(feature_names)
            else:
                # Otherwise, just append the feature names as is
                output_features.extend(features)
        else:
            # If the remainder transformer is used, handle accordingly
            remainder_features = [f for f in features if f not in output_features]
            output_features.extend(remainder_features)

    return output_features



# Separar en conjuntos de train, validation y test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=(1/3), random_state=42)


date_transformer = FunctionTransformer(extract_date_parts, validate=False)

# Crear un ColumnTransformer para procesar los datos


numeric_features = df.select_dtypes(include=['int64', 'float64']).drop('quantity', axis=1).columns

# Lista inicial de características categóricas
categorical_features = ['day', 'month', 'year']

# Añadir columnas del DataFrame que no son de tipo int64 o float64
categorical_features.extend(df.select_dtypes(exclude=['int64', 'float64']).columns)

# Excluir la columna 'date'
categorical_features.remove('date')

# Creando un dataframe que solo contiene la columna 'quantity'
train_Y = train_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
train_X = train_df.drop(columns=['quantity'])

#print(train_X.columns)

# Creando un dataframe que solo contiene la columna 'quantity'
test_Y = test_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
test_X = test_df.drop(columns=['quantity'])

# Creando un dataframe que solo contiene la columna 'quantity'
val_Y= val_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
val_X = val_df.drop(columns=['quantity'])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False), categorical_features)
    ])



# Crear y entrenar el pipeline con DummyRegressor
pipeline_dummy = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


#print("debug1")
pipeline_dummy.fit(train_X, train_Y) 

# Evaluar el modelo
y_pred = pipeline_dummy.predict(val_X)

mae_dummy = mean_absolute_error(val_Y, y_pred)
print(f'MAE con DummyRegressor: {mae_dummy}')


# After fitting your pipeline, call this function
feature_names_after_preprocessing = get_feature_names(pipeline_dummy.named_steps['preprocessor'])
print("Features after preprocessing:", feature_names_after_preprocessing)


# Reemplazar DummyRegressor con XGBRegressor y entrenar nuevamente
pipeline_xgb = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(train_X, train_Y) 

# Evaluar el nuevo modelo
y_pred_xgb = pipeline_xgb.predict(val_X)
mae_xgb = mean_absolute_error(val_Y, y_pred_xgb)
print(f'MAE con XGBRegressor: {mae_xgb}')

# Guardar los modelos
joblib.dump(pipeline_dummy, 'model_dummy.pkl')
joblib.dump(pipeline_xgb, 'model_xgb.pkl')

MAE con DummyRegressor: 13298.497767341096
Features after preprocessing: ['price', 'lat', 'long', 'pop', 'day_28', 'day_29', 'day_30', 'day_31', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'city_Athens', 'city_Irakleion', 'city_Larisa', 'city_Patra', 'city_Thessaloniki', 'shop_shop_1', 'shop_shop_2', 'shop_shop_3', 'shop_shop_4', 'shop_shop_5', 'shop_shop_6', 'brand_adult-cola', 'brand_gazoza', 'brand_kinder-cola', 'brand_lemon-boost', 'brand_orange-power', 'container_can', 'container_glass', 'container_plastic', 'capacity_1.5lt', 'capacity_330ml', 'capacity_500ml']
MAE con XGBRegressor: 2432.32102527273


['model_xgb.pkl']

### Respuestas

5. El MAE mide el promedio de los errores absolutos entre las predicciones y los valores reales, reflejando la diferencia media entre la cantidad de bebidas vendidas pronosticada por el modelo y la cantidad realmente vendida. Un MAE más bajo indica una mayor precisión en las predicciones. En términos empresariales, un MAE bajo sugiere que las predicciones del modelo están cercanas a los valores reales, lo que permite a la empresa planificar mejor la producción y la distribución, minimizando los costos asociados a excesos o faltas de inventario.

6. Al comparar el desempeño del XGBRegressor con el DummyRegressor, se observa una mejora significativa al emplear el XGBRegressor. Mientras que el DummyRegressor ofrece un MAE de 13298.5, el XGBRegressor reduce esta métrica a 2432.3, lo que indica una precisión notablemente superior. Esto implica que el XGBRegressor, incluso con parámetros por defecto, es capaz de capturar mejor las tendencias y patrones en los datos, proporcionando pronósticos más precisos y útiles para la toma de decisiones empresariales. Esta mejora en la precisión es crucial para una gestión eficaz de la cadena de suministro y para satisfacer la demanda del mercado de manera más efectiva.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [59]:
# Definir los constraints de monotonía
# Asumiendo que 'price' es la primera columna después del preprocesamiento
# -1 indica una relación monótona negativa
monotone_constraints = (-1,) + (0,) * (len(feature_names_after_preprocessing) - 1)

print(monotone_constraints)
# Incluir el inspector de datos en el pipeline antes del regresor
pipeline_xgb_monotone = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotone_constraints))
])

pipeline_xgb_monotone.fit(train_X, train_Y)

# Evaluar el modelo
y_pred_xgb_monotone = pipeline_xgb_monotone.predict(val_X)
mae_xgb_monotone = mean_absolute_error(val_Y, y_pred_xgb_monotone)
print(f'MAE con XGBRegressor y constraint monótono: {mae_xgb_monotone}')

# Guardar el modelo
joblib.dump(pipeline_xgb_monotone, 'model_xgb_monotone.pkl')

(-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


MAE con XGBRegressor y constraint monótono: 2485.6526758947443


['model_xgb_monotone.pkl']

### Respuesta

Tras aplicar esta restricción, se observa un cambio en el mean_absolute_error (MAE), que pasa de 2432.3 a 2485.7. Este ligero aumento sugiere que, aunque la relación inversa entre precio y cantidad es intuitivamente plausible, su incorporación directa en el modelo no necesariamente mejora la precisión de las predicciones en este caso específico. Este resultado puede indicar que la relación entre precio y demanda no es tan directa o que otros factores no considerados en el modelo pueden estar influyendo en la demanda. Por lo tanto, aunque la intuición del colega era razonable, la realidad de los datos indica que la relación entre precio y demanda puede ser más compleja de lo inicialmente sugerido. Este hallazgo subraya la importancia de validar suposiciones teóricas con datos reales y ajustar los modelos en consecuencia.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [60]:
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)


# Define la función objetivo para Optuna
def objective(trial):
    # Definir los rangos de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Crear un nuevo pipeline con los hiperparámetros sugeridos por Optuna
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state = 314159
    )

    # Modificar el valor de min_frequency del OneHotEncoder en el ColumnTransformer
    for name, transformer, columns in preprocessor.transformers_:
        if isinstance(transformer, OneHotEncoder):
            transformer.set_params(min_frequency=min_frequency)

    pipeline_xgb_optuna = Pipeline(steps=[
        ('date', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb_model)
    ])

    pipeline_xgb_optuna.fit(train_X, train_Y)

    # Calcular MAE en datos de validación
    y_pred = pipeline_xgb_optuna.predict(val_X)
    mae = mean_absolute_error(val_Y, y_pred)
    
    # Guardar el modelo en los atributos de usuario del mejor intento
    trial.set_user_attr('model', xgb_model)

    return mae

# Crear un estudio de Optuna
study_opt = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=314159))
study_opt.optimize(objective, n_trials=100, timeout=300)

# Obtener los mejores hiperparámetros encontrados
best_params = study_opt.best_params
mae_optuna_1 = study_opt.best_value
num_trials = len(study_opt.trials)

print("Número de trials:", num_trials)
print("Mejores hiperparámetros:", best_params)
print("MAE óptimo:", mae_optuna_1)



[I 2023-11-17 22:34:57,655] A new study created in memory with name: no-name-3dc4852a-c521-40c7-898c-541fe8a2a9d7


[I 2023-11-17 22:35:01,323] Trial 0 finished with value: 2678.4733948941043 and parameters: {'learning_rate': 0.08197440749175473, 'n_estimators': 574, 'max_depth': 6, 'max_leaves': 9, 'min_child_weight': 5, 'reg_alpha': 0.9673564038996015, 'reg_lambda': 0.09820669376309132, 'min_frequency': 0.8018603712796477}. Best is trial 0 with value: 2678.4733948941043.
[I 2023-11-17 22:35:02,034] Trial 1 finished with value: 2112.7636541907696 and parameters: {'learning_rate': 0.060885309625463256, 'n_estimators': 606, 'max_depth': 6, 'max_leaves': 48, 'min_child_weight': 2, 'reg_alpha': 0.30960447319461515, 'reg_lambda': 0.9079540077466299, 'min_frequency': 0.18498483412439337}. Best is trial 1 with value: 2112.7636541907696.
[I 2023-11-17 22:35:02,322] Trial 2 finished with value: 4393.54624884725 and parameters: {'learning_rate': 0.019635701927839904, 'n_estimators': 134, 'max_depth': 9, 'max_leaves': 64, 'min_child_weight': 5, 'reg_alpha': 0.5330602987538661, 'reg_lambda': 0.8794020557461902

Número de trials: 100
Mejores hiperparámetros: {'learning_rate': 0.07841424236482403, 'n_estimators': 982, 'max_depth': 8, 'max_leaves': 100, 'min_child_weight': 3, 'reg_alpha': 0.4224399066073921, 'reg_lambda': 0.966078887578785, 'min_frequency': 0.6384252527728932}
MAE óptimo: 1921.1069859914696


In [61]:
# Después de la optimización, antes de guardar el objeto Study
best_model = study_opt.best_trial.user_attrs.get('model')

# Guardar el objeto Study original
if best_model:
    study_opt.best_trial.user_attrs['model'] = best_model
    joblib.dump(study_opt, 'model_xgb_optuna.pkl')
else:
    print("No se encontró el modelo en los atributos de usuario del mejor intento.")

#### Hiperparámetros:

- learning_rate: es un hiperparámetro que controla la tasa de aprendizaje del modelo. En este sentido, si la tasa de aprendizaje es muy alta, el modelo aprende rápidamente, pero hace que este sea inestable, mientras que una tasa de aprendizaje baja hace que el modelo aprenda más lentamente, pero de una forma más precisa. En este caso se plantea que varíe entre los decimales 0.001 y 0.1 los cuales son considerados como tasas de aprendizajes relativamente baja y alta respectivamente.
- n_estimators: indica la cantidad de árboles en el bosque de XGBoost. Una mayor cantidad de árboles puede mejorar la capacidad del modelo para capturar las relaciones que son más complejas entre las variables, pero al mismo tiempo, al haber más estimadores también aumenta el tiempo de entrenamiento del modelo. En este caso el valor de n_estimator varía entre los enteros 50 y 1000, lo cual es un buen rango para optimizar el modelo.
- max_depth: indica la profundidad máxima que pueden alcanzar los árboles generados por el modelo. Una profundidad muy alta puede hacer que el árbol se sobreajuste a los datos. Este valor varía entre los enteros 3 y 10, lo cual es un buen rango para optimizar el modelo.
- max_leaves: indica el número máximo de hojas de cada árbol, restringiendo así la complejidad del mismo. Este valor varía entre los enteros 0 y 100, lo cual es un buen rango para optimizar el modelo.
- min_child_weight: indica el peso mínimo (número de instancias) que debe tener un nodo para generar una nueva división del mismo. Un mayor valor de este hiperparámetro puede restringir el que se produzcan más nodos. Este valor varía entre los entre los enteros de 1 a 5, lo cual es un buen rango para optimizar el modelo.
- reg_alpha: indica el nivel de regularización de Lasso a las variables. En este sentido, puede eliminar variables que se vean afectadas por la regularización. Este valor varía en decimales entre 0 y 1, lo cual es un buen rango para optimizar el modelo.
- reg_lambda: indica el nivel de regularización de Ridge a las variables. En este sentido, evita que los coeficientes de las variables tomen valores muy altos, pero no elimina variables de la ecuación. Este valor varía en decimales entre 0 y 1, lo cual es un buen rango para optimizar el modelo.
- OneHotEncoder (min_frequency): indica el mínimo de repeticiones que debe tener una categoría para que sea considerada en la codificación. En este sentido, el valor entregado varía entre decimalmente entre 0 y 1. Estos valores entregados al algoritmo son incorrectos, pues no tiene sentido que se entregue valores decimales entre 0 y 1 a min_frequency, cuando esta debe recibir valores enteros mayores a 0.


Con respecto al mejor resultado anterior a Optuna, donde se obtenía un MAE de 2432 en la actividad 1.1, es decir, en el modelo sin relaciones forzadas, ahora se logra un MAE de 1921, lo que se traduce en una mejora cercana al 21%. Esto se debe a que optuna logra encontrar aquellos hiperparámetros en los que mejor se comporta el modelo en el tiempo que se le da. 

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [62]:
import optuna
from optuna.integration import XGBoostPruningCallback
import time
from IPython.display import clear_output

# Callback para actualizar la salida después de cada trial
def print_callback(study, trial):
    clear_output(wait=True)
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}.")
    print(f"Best is trial {study.best_trial.number} with value: {study.best_value}.")


# Define la función objetivo para Optuna con Prunning
def objective_with_prunning(trial):
    # Definir los rangos de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Ajustar el valor de min_frequency del OneHotEncoder en el ColumnTransformer
    for name, transformer, columns in preprocessor.transformers_:
        if isinstance(transformer, OneHotEncoder):
            transformer.set_params(min_frequency=min_frequency)

    # Crear un pipeline solo para el preprocesamiento
    pipeline_preprocess = Pipeline(steps=[
        ('date', date_transformer),
        ('preprocessor', preprocessor)
    ])

    # Transformar el conjunto de validación
    X_valid_transformed = pipeline_preprocess.transform(val_X)

    # Crear el modelo XGBRegressor con los hiperparámetros sugeridos por Optuna
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda
    )

    # Implementar Prunning
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")

    # Entrenar el modelo con los datos de entrenamiento transformados y validación
    xgb_model.fit(pipeline_preprocess.transform(train_X), train_Y, 
                  eval_set=[(X_valid_transformed, val_Y)],
                  early_stopping_rounds=10,
                  callbacks=[pruning_callback])

    # Calcular MAE en datos de validación transformados
    y_pred = xgb_model.predict(X_valid_transformed)
    mae = mean_absolute_error(val_Y, y_pred)

    # Guardar el modelo en los atributos de usuario del mejor intento
    trial.set_user_attr('model', xgb_model)
    
    return mae



# Crear un estudio de Optuna con Prunning
optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='xgboost.sklearn')
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=314159))
study.optimize(objective_with_prunning, n_trials=100, timeout=300, show_progress_bar=True, callbacks=[print_callback])

# Obtener los mejores hiperparámetros encontrados
best_params = study.best_params
mae_optuna_2 = study.best_value
num_trials = len(study.trials)


print("Número de trials con Prunning:", num_trials)
print("Mejores hiperparámetros con Prunning:", best_params)
print("MAE óptimo con Prunning:", mae_optuna_2)

# Después de la optimización, antes de guardar el objeto Study
best_model = study_opt.best_trial.user_attrs.get('model')

# Guardar el objeto Study original
if best_model:
    study_opt.best_trial.user_attrs['model'] = best_model
    joblib.dump(study_opt, 'study_prunning.pkl')
else:
    print("No se encontró el modelo en los atributos de usuario del mejor intento.")

Trial 99 finished with value: 16597.418012698825 and parameters: {'learning_rate': 0.08635476748128637, 'n_estimators': 797, 'max_depth': 10, 'max_leaves': 82, 'min_child_weight': 3, 'reg_alpha': 0.12669047868897476, 'reg_lambda': 0.9826611469453277, 'min_frequency': 0.8321880030439613}.
Best is trial 78 with value: 1932.1304387195569.


Best trial: 78. Best value: 1932.13: 100%|██████████| 100/100 [04:14<00:00,  2.54s/it, 253.95/300 seconds]


Número de trials con Prunning: 100
Mejores hiperparámetros con Prunning: {'learning_rate': 0.09644971102145625, 'n_estimators': 706, 'max_depth': 9, 'max_leaves': 87, 'min_child_weight': 3, 'reg_alpha': 0.08846699130326877, 'reg_lambda': 0.8223771364146057, 'min_frequency': 0.6659720413708433}
MAE óptimo con Prunning: 1932.1304387195569


Prunning se refiere a la eliminación de algunas de las ramas de los árboles de decisión contenidos en el algoritmo de XGB con la finalidad de reducir la complejidad y el sobreajuste del modelo.

Con respecto al modelo de la pregunta 1.3, este modelo obtiene resultados levemente peores de acuerdo con el MAE, pasando de 1921 a 1932. Una posible razón de por qué pasa esto es que el algoritmo de prunning haya eliminado ramas del árbol que eran útiles para reducir el error. Una forma de mejorar estos resultados podría ser aumentar el número de rondas, usando además un early stopping para evitar el sobreajuste del modelo.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [63]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study_opt)


Para el mejor modelo entrenado, es decir, el primero de optuna, el gráfico permite ver que la primera gran mejora ocurre del primer al segundo trial, para luego obtener una nueva gran mejora entre los trials 15 y 17. Luego de estos trials el modelo continúa mejorando pero de forma más leve.

In [64]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study_opt)


A partir del gráfico de tendencias paralelas, se pueden ver tendencias en función de los cambios en los hiperparámetros del modelo:
- Learning rate: se puede ver que la mayoría de los trials fluyen a través de learning rates entre 0.06 y 0.099.
- max_depth: para este hiperparámetro no se ve una tendencia clara, pero si se puede notar que  los valores de 7, 8, 9 y 10 tienen una mayor cantidad de trials que los utilizan.
- max_leaves: en este caso se observa que hay mayor tendencia a valores mayores a 80 en este hiperparámetro.
- n_estimators: aquí también se puede encontrar una tendencia donde la mayoría de los trials toman valores entre 800 y 1000 para este hiperparámetro.

Fuera de estos hiperparámetros no se observan tendencias claras. De todas maneras, al ser tan sutiles las mejoras entre trials, no es posible notar qué trial corresponde a los mejores modelos ya que el cambio en el rango de color es mínimo.

In [65]:
from optuna.visualization import plot_param_importances

plot_param_importances(study_opt)


El claro hiperparámetro con mayor importancia es max_leaves. Luego, pero de menor forma, se encuentran n_estimators y max_depth.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [66]:
# Calcula el MAE para cada modelo
mae_baseline = mae_dummy
mae_xgboost = mae_xgb  
mae_optuna = mae_optuna_1
mae_constraints = mae_xgb_monotone 
mae_pruning = mae_optuna_2

# Crea un DataFrame con los valores de MAE
data = {
    'Modelo': ['Baseline', 'XGBoost', 'XGBoost (Optuna)', 'XGBoost (Constraints)', 'XGBoost (Optuna, Pruning)'],
    'MAE': [mae_baseline, mae_xgboost, mae_optuna, mae_constraints, mae_pruning]
}

tabla = pd.DataFrame(data)

# Imprime la tabla resumen
display(tabla)

,Modelo,MAE
0,Baseline,13298.497767
1,XGBoost,2432.321025
2,XGBoost (Optuna),1921.106986
3,XGBoost (Constraints),2485.652676
4,"XGBoost (Optuna, Pruning)",1932.130439


De acuerdo con la tabla que muestra el mejor MAE por modelo entrenado, el mejor modelo corresponde al modelo que utiliza optuna sin prunning, es decir el de la parte 1.3, pues obtiene el valor más cercano a 0 de todos los modelos.

In [67]:
loaded_model = joblib.load('model_xgb_optuna.pkl')

In [68]:
# Inspeccionar las claves en los atributos de usuario del mejor intento
print("Claves de atributos de usuario:", loaded_model.best_trial.user_attrs.keys())


Claves de atributos de usuario: dict_keys(['model'])


In [69]:
# Obtener el mejor modelo del objeto Study
best_model = loaded_model.best_trial.user_attrs['model']

# Aplicar el preprocesamiento al conjunto de prueba
pipeline_pre = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor)
])

test_X = pipeline_pre.fit_transform(test_X)

# Hacer predicciones en el conjunto de prueba
test_predictions = best_model.predict(test_X)

# Calcular el MAE en el conjunto de prueba
mae_test = mean_absolute_error(test_Y, test_predictions)


In [70]:
print(mae_test)

3404.5347704030874


El MAE sobre los datos de test es de 3404, un valor de casi el doble al obtenido sobre los datos de validación de 1921. Esto puede deberse a que existan diferencias importantes entre los datos de validación y los datos de testeo. Además, existe la posibilidad de que si los datos de validación no representan completamente la variabilidad de los datos, los hiperparámetros que encontró optuna estén sesgados hacia estos datos, lo que produce que cuando se presentan datos "nuevos" el modelo no obtenga los mismos resultados esperados que para el dataset de validación.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>